In [5]:
import os
import torch
import torch.optim as optim

# Import our model and trainer for VQ-VAE
from models.conv_vqvae_model import ConvVQVAE, vqvae_loss_function, print_num_params
from trainer.trainer_vqvae import VQVAETrainer
from dataloader.dataloader import load_fraud_data, load_config

# Build the config path relative to this notebook's location
config_path = "configs/conv_vqvae/fraud_conv_vqvae.config"

# Use configparser to maintain compatibility with your model code
import configparser
config_parser = configparser.ConfigParser()
config_parser.read(config_path)

# Extract the model configuration from the "Conv_VQVAE" section
conv_vqvae_config = config_parser["Conv_VQVAE"]

# Use load_config for the dataloader (it expects the dictionary format)
config_dict = load_config(config_path)

# Load data (this will override defaults with settings from the config's DataLoader section)
data = load_fraud_data(config_path=config_path)
dataloaders = data['dataloaders']
input_dim = data['input_dim']

# Instantiate the VQ-VAE model using the config section
model = ConvVQVAE(conv_vqvae_config)
print_num_params(model)

# Retrieve training hyperparameters from the Trainer section (including learning rate)
lr = config_parser["Trainer"].getfloat("lr")
num_epochs = config_parser["Trainer"].getint("num_epochs")

# Define the optimizer and loss function for the VQ-VAE
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = vqvae_loss_function

# Create the trainer instance
trainer = VQVAETrainer(model, dataloaders, loss_fn, optimizer)

# Train the model (this will use tqdm for progress reporting)
train_losses, val_losses = trainer.train(num_epochs=num_epochs)

Loaded configuration from configs/conv_vqvae/fraud_conv_vqvae.config
Filtered dataset to class 1: 378 samples
Normalization statistics (calculated from class 1): {'Time': {'mean': 80790.48148148147, 'std': 48332.5139872635}, 'Amount': {'mean': 133.6764814814815, 'std': 276.3532237447719}}
Filtered dataset to class 1: 378 samples
Filtered dataset to class 1: 47 samples
Filtered dataset to class 1: 48 samples
Total number of trainable parameters: 64164


Epoch 1/50: Train Loss = 696.5127, Val Loss = 1022.2650


Epoch 2/50: Train Loss = 675.0035, Val Loss = 1008.6584


Epoch 3/50: Train Loss = 690.4520, Val Loss = 1009.6366


Epoch 4/50: Train Loss = 663.4775, Val Loss = 1032.1196


...

Epoch 46/50: Train Loss = 668.4558, Val Loss = 1052.4761


Epoch 47/50: Train Loss = 667.5610, Val Loss = 1023.2524


Epoch 48/50: Train Loss = 661.3401, Val Loss = 1006.8302


Epoch 49/50: Train Loss = 659.1544, Val Loss = 1011.3263


Epoch 50/50: Train Loss = 662.4793, Val Loss = 1026.8474
